In [88]:
import sqlite3
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [89]:
db= "../input/soccer/database.sqlite"

In [90]:
conn = sqlite3.connect(db)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [91]:
countries= pd.read_sql("""SELECT DISTINCT * from Country""", conn)
countries

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


In [92]:
team_league= pd.read_sql("""SELECT * FROM Country 
                            JOIN League ON League.id= Country.id """ , conn)
team_league

,id,name,id,country_id,name
0,1,Belgium,1,1,Belgium Jupiler League
1,1729,England,1729,1729,England Premier League
2,4769,France,4769,4769,France Ligue 1
3,7809,Germany,7809,7809,Germany 1. Bundesliga
4,10257,Italy,10257,10257,Italy Serie A
5,13274,Netherlands,13274,13274,Netherlands Eredivisie
6,15722,Poland,15722,15722,Poland Ekstraklasa
7,17642,Portugal,17642,17642,Portugal Liga ZON Sagres
8,19694,Scotland,19694,19694,Scotland Premier League
9,21518,Spain,21518,21518,Spain LIGA BBVA


In [93]:
#Ages of players
ages=pd.read_sql("""SELECT player_name, (strftime('%Y', 'now') - strftime('%Y', birthday)) AS age
                    FROM Player 
                    ORDER BY age""",conn)
ages

,player_name,age
0,Alban Lafont,22
1,Gianluigi Donnarumma,22
2,Jonathan Leko,22
3,Christian Pulisic,23
4,Christopher McLaughlin,23
...,...,...
11055,Luca Bucci,52
11056,Michael Tarnat,52
11057,Rob van Dijk,52
11058,Paolo Maldini,53


In [97]:
#Average rating and potentials by age distribution 
Avg= pd.read_sql(""" SELECT STRFTIME("%Y",birthday) AS BD_year , 
                        CASE
                        WHEN CAST(STRFTIME("%Y",birthday) AS INT) BETWEEN 1967 AND 1977 THEN 'Older Player'
                        WHEN CAST(STRFTIME("%Y",birthday) AS INT) BETWEEN 1978 AND 1987 THEN 'Medium Age Player'
                        WHEN CAST(STRFTIME("%Y",birthday) AS INT)>1987 THEN 'Younger Player'
                        ELSE 'Other' 
                        END AS Ages,
                        COUNT('BD_year') AS Distribution,
                        (AVG(P.avg_overall_rating)) AS AVG_rating,
                        (AVG(P.avg_potential)) AS AVG_potential
                        FROM PLAYER
                        LEFT JOIN (SELECT Player_Attributes.player_api_id, 
                                        AVG(Player_Attributes.overall_rating) AS avg_overall_rating,
                                        AVG(Player_Attributes.potential) AS avg_potential  
                                        FROM Player_Attributes
                                        GROUP BY Player_Attributes.player_api_id) 
                                        AS P ON PLAYER.player_api_id = P.player_api_id
                    GROUP BY BD_year;
                    """, conn )
Avg

,BD_year,Ages,Distribution,AVG_rating,AVG_potential
0,1967,Older Player,1,76.000000,77.250000
1,1968,Older Player,1,79.500000,88.500000
2,1969,Older Player,6,70.259259,77.529630
3,1970,Older Player,12,71.842395,76.796959
4,1971,Older Player,12,71.902165,76.913546
5,1972,Older Player,22,71.122984,75.979408
6,1973,Older Player,43,72.783384,78.671515
7,1974,Older Player,77,69.657200,75.476657
8,1975,Older Player,113,70.691449,74.176188
9,1976,Older Player,156,70.750483,74.052488


In [95]:
#List of matches and number of goals
matches=pd.read_sql(""" SELECT Match.id, 
                        Country.name AS country_name, League.name AS league_name,season, HT.team_long_name AS home_team, AT.team_long_name AS away_team,
                        home_team_goal, away_team_goal
                        FROM Match 
                        JOIN Country ON Country.id=Match.country_id
                        JOIN League On League.id=Match.league_id
                        JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                        JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                        WHERE country_name= 'France'
                        ORDER BY date""", conn)
matches

,id,country_name,league_name,season,home_team,away_team,home_team_goal,away_team_goal
0,4769,France,France Ligue 1,2008/2009,AJ Auxerre,FC Nantes,2,1
1,4770,France,France Ligue 1,2008/2009,Girondins de Bordeaux,SM Caen,2,1
2,4771,France,France Ligue 1,2008/2009,Le Havre AC,OGC Nice,1,0
3,4772,France,France Ligue 1,2008/2009,Le Mans FC,FC Lorient,0,1
4,4774,France,France Ligue 1,2008/2009,AS Monaco,Paris Saint-Germain,1,0
...,...,...,...,...,...,...,...,...
3035,7744,France,France Ligue 1,2015/2016,FC Lorient,GFC Ajaccio,1,0
3036,7745,France,France Ligue 1,2015/2016,Paris Saint-Germain,FC Nantes,4,0
3037,7746,France,France Ligue 1,2015/2016,SM Caen,Girondins de Bordeaux,1,0
3038,7747,France,France Ligue 1,2015/2016,Stade de Reims,Olympique Lyonnais,4,1


In [96]:
#average home and away goals of each countries 
avg_goals=pd.read_sql("""SELECT Country.name AS country_name,
                            AVG(home_team_goal) AS avg_home_team_goals,
                            AVG(away_team_goal) AS avg_away_team_goals,
                            SUM(home_team_goal+away_team_goal) AS total_goals
                        FROM Match
                        JOIN Country on Country.id= Match.country_id
                        GROUP BY Country.name
                        ORDER BY total_goals DESC  """, conn)
avg_goals

,country_name,avg_home_team_goals,avg_away_team_goals,total_goals
0,Spain,1.631250,1.135855,8412
1,England,1.550987,1.159539,8240
2,Italy,1.500829,1.116009,7895
3,Netherlands,1.779820,1.301062,7542
4,France,1.402961,1.040132,7427
5,Germany,1.626634,1.274918,7103
6,Portugal,1.408382,1.126218,5201
7,Belgium,1.609375,1.192130,4841
8,Scotland,1.429276,1.204496,4804
9,Poland,1.394792,1.030208,4656
